# PyFP - FP Demystified

### Helpful Resources
* https://www.youtube.com/watch?v=p9_r36fIrqc
* https://github.com/3kwa/frisby/blob/master/presentation.ipynb
* frisby.py

# Functors

In [1]:
def next_char_from_string(s):
    strip = s.strip()
    cast = int(strip)
    inc = cast + 1
    return chr(inc)

next_char_from_string(' 64 ')

'A'

In [2]:
def next_char_from_string_2(s):
    return chr(int(s.strip())+1)

next_char_from_string_2('   64  ')

'A'

# Implementing Box and Folding

In [3]:
from frisby import Box

In [4]:
def identity(x): return x

In [5]:
identity = lambda x : x

In [6]:
Box.of(1)

Box(1)

In [7]:
Box.of(1).map(lambda x: x + 1)

Box(2)

In [8]:
Box.of(1).map(lambda x: x + 1).map(lambda x: x ** 2)

Box(4)

In [9]:
Box.of(1).map(lambda x: x + 1).map(lambda x: x ** 2).fold(identity)

4

In [10]:
Box.of(1).map(lambda x: x + 1).fold(lambda x: x ** 2)

4

In [11]:
def next_char_from_string_3(s):
    return (
            Box.of(s)
            .map(lambda s: s.strip())
            .map(int)
            .map(lambda x: x + 1)
            .fold(chr)
            )

next_char_from_string_2('   64  ')

'A'

# Currying and partial functools

In [12]:
def add(x, y):
    return x + y

def inc(y):
    # a specialized add
    return add(1, y)

add(1, 3), inc(3)

(4, 4)

## functools partial

In [13]:
from functools import partial

inc = partial(add, 1)

add(1, 3), inc(3)

(4, 4)

## curried function

In [14]:
def add(x):
    def inner(y):
        return x + y
    return inner

inc = add(1)
add(1)(3), inc(3)

(4, 4)

## lambda expressions

In [15]:
add = lambda x: lambda y: x + y
inc = add(1)

add(1)(3), inc(3)

(4, 4)

## Example from the real world

In [16]:
import re

In [17]:
replace = lambda pattern: lambda repl: lambda string: \
    re.sub(pattern, repl, string)

censor = replace(r"[aeiou]")("*")
print(censor("PyCon AU"))

map_ = lambda f: lambda iterable: list(map(f, iterable))

censor_all = map_(censor)

print(censor_all(["PyCon AU","Sunday sesh"]))

PyC*n AU
['PyC*n AU', 'S*nd*y s*sh']


# Applicative Functors
Should look something like this
```
F.of(f).apply(F.of(x)) = F.of(x).map(f)
```

In [18]:
def square(x): return x ** 2

In [19]:
Box.of(2).map(square)

Box(4)

In [20]:
Box.of(square).apply(Box.of(2))

Box(4)

In [21]:
add = lambda x: lambda y: x + y

Box.of(add).apply(Box.of(2)).apply(Box.of(3))

Box(5)

In [22]:
def lift_a2(f, fx, fy):
    return fx.map(f).apply(fy)

lift_a2(add, Box.of(3), Box.of(4))

Box(7)

# Either Left or Right

You can go either right or left. Used with disjunction

In [23]:
from frisby import Left, Right

In [24]:
Right(3).map(lambda i: i + 1).map(lambda i: i / 2)

Right(2.0)

In [25]:
Left(3).map(lambda i: i + 1).map(lambda i: i / 2)

Left(3)

In [26]:
import random

either = random.choice([Left(3), Right(3)])

(
    either
    .map(lambda i: i + 1)
    .map(lambda i: i / 2)
#    .fold(...)
)

Right(2.0)

In [27]:
import random

either = random.choice([Left(3), Right(3)])

(
    either
    .map(lambda i: i + 1)
    .map(lambda i: i / 2)
    .fold(lambda e: "taebenacle", identity)
)

'taebenacle'

In [28]:
from frisby import from_nullable

In [29]:
from_nullable??

In [30]:
known_colors = {"red": "#ff4444",
                "blue": "#3b5998",
                "yellow": "#fff68f"}

In [31]:
def upper_case_hex_of(color):
    """
    >>> upper_case_hex("blue")
    3B5998
    """
    return (
           from_nullable(known_colors.get(color))
            .map(lambda s: s[1:])
            .fold(lambda e: 'unknown',
                  lambda s: s.upper())
           )

upper_case_hex_of("blue")#, upper_case_hex_of("green")

'3B5998'

In [32]:
from frisby import try_except

try_except??

In [33]:
def read(filename):
    """
    >>> read("config.json")
    '{"port": 8888}\\n'
    """
    with open(filename) as f:
        return f.read()

In [34]:
import json

In [35]:
config = "{\"port\": 8888}"
json.loads(config).get('port', 3000)

8888

In [36]:
(
    try_except(lambda: read("config.json"))
    .map(lambda s: try_except(lambda: kson.loads(s)))
)

Left([Errno 2] No such file or directory: 'config.json')

In [37]:
(
    try_except(lambda: read("config.json"))
    .chain(lambda s: try_except(lambda: kson.loads(s)))
    .fold(lambda e: 3000,
         lambda c: c.get('port',3000))
)

3000

# Monad

In [38]:
from frisby import Monadic

Monadic??

# SemiGroup

In [39]:
from frisby import SemiGroup

SemiGroup??

# Monoid

In [40]:
from frisby import Monoid

Monoid??

In [41]:
class Sum(Monoid):
    
    def concat(self, other):
        return Sum(self.x + other.x)
    
    @classmethod
    def empty(cls):
        return cls(0)

Sum(1).concat(Sum(2)).concat(Sum(3).concat(Sum.empty()))


Sum(6)